In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.4.1'

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

In [9]:
ann = tf.keras.models.Sequential()

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='selu'))

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='selu'))

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='selu'))

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [15]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5442 - accuracy: 0.7967
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4623 - accuracy: 0.8013
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4474 - accuracy: 0.7988
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4369 - accuracy: 0.7977
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4321 - accuracy: 0.7983
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4223 - accuracy: 0.8017
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4135 - accuracy: 0.7970
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4053 - accuracy: 0.7976
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3933 - accuracy: 0.8006
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3948 - accura

In [18]:
y_pred = ann.predict(X_test)
y_pred

array([[0.31457406],
       [0.28985584],
       [0.13528332],
       ...,
       [0.11684862],
       [0.14031982],
       [0.18536466]], dtype=float32)

In [19]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1497   98]
 [ 179  226]]


In [21]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

In [22]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [23]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)

In [24]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [25]:
mean = accuracies.mean()

In [26]:
variance = accuracies.std()

In [27]:
from keras.layers import Dropout

In [28]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [49]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean, variance)

nan nan


In [30]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [31]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [32]:
classifier = KerasClassifier(build_fn = build_classifier)


In [38]:
parameters = {'batch_size': [10, 25, 32],
              'epochs': [50, 100],
              'optimizer': ['adam', 'rmsprop']}

In [39]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [40]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [41]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 1s 1ms/step - loss: 0.5441 - accuracy: 0.8035
Epoch 2/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4313 - accuracy: 0.8085
Epoch 3/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4360 - accuracy: 0.8089
Epoch 4/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4355 - accuracy: 0.8107
Epoch 5/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4336 - accuracy: 0.8090
Epoch 6/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4319 - accuracy: 0.8075
Epoch 7/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4339 - accuracy: 0.8112
Epoch 8/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4250 - accuracy: 0.8153
Epoch 9/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4275 - accuracy: 0.8112
Epoch 10/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4344 - accuracy: 0.8049

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


A saída de streaming foi truncada nas últimas 5000 linhas.
Epoch 1/50
225/225 [==============================] - 2s 3ms/step - loss: 0.6360 - accuracy: 0.7855
Epoch 2/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4416 - accuracy: 0.8028
Epoch 3/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4400 - accuracy: 0.8036
Epoch 4/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4435 - accuracy: 0.8049
Epoch 5/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4369 - accuracy: 0.8143
Epoch 6/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4304 - accuracy: 0.8124
Epoch 7/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4309 - accuracy: 0.8160
Epoch 8/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4394 - accuracy: 0.8099
Epoch 9/50
225/225 [==============================] - 1s 2ms/step - loss: 0.4264 - accuracy: 0.8177
Epoch 10/50
225/225 [====================

In [46]:
best_parameters = grid_search.best_params_
best_acuracy = grid_search.best_score_ 

In [48]:
print('Best parameters: \n',best_parameters,'\nBest acuracy: \n', best_acuracy)

Best parameters: 
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'} 
Best acuracy: 
 0.8520000000000001
